In [2]:
from dotenv import dotenv_values
from utils.slovenian_preprocessor import Slovenian_preprocessor
import pandas as pd
HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']

data_train = pd.read_csv('/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/data/slovenian/E3C_Slovenian_Train_SL_L1.csv', header=None, names=['word', 'label'])
preprocessor = Slovenian_preprocessor(data_train, "meta-llama/Llama-2-7b-chat-hf", "meta-llama/Llama-2-7b-chat-hf", token_llama=HF_TOKEN)
preprocessor.preprocess()
preprocessor.apply('', offset=False, simplest_prompt=False)
preprocessor.data[0]['prompt']

MODEL TYPE: llama


Map: 100%|██████████| 668/668 [00:00<00:00, 22030.33 examples/s]


'<s>[INST]  <<61 - letni moški se je pojavil s kroničnim levim ušesom izcedekom , glavobolom na levi strani , diplopijo povezano z vrtoglavico , tinitusom in slušno okvaro .>>> [/INST][{"entity": "levim ušesom izcedekom"}, {"entity": "glavobolom na levi strani"}, {"entity": "diplopijo"}, {"entity": "vrtoglavico"}, {"entity": "tinitusom"}, {"entity": "slušno okvaro"}] </s>'

In [3]:
preprocessor.data[0]

{'text': '61 - letni moški se je pojavil s kroničnim levim ušesom izcedekom , glavobolom na levi strani , diplopijo povezano z vrtoglavico , tinitusom in slušno okvaro .',
 'entities': [{'entity': 'levim ušesom izcedekom'},
  {'entity': 'glavobolom na levi strani'},
  {'entity': 'diplopijo'},
  {'entity': 'vrtoglavico'},
  {'entity': 'tinitusom'},
  {'entity': 'slušno okvaro'}],
 'prompt': '<s>[INST]  <<61 - letni moški se je pojavil s kroničnim levim ušesom izcedekom , glavobolom na levi strani , diplopijo povezano z vrtoglavico , tinitusom in slušno okvaro .>>> [/INST][{"entity": "levim ušesom izcedekom"}, {"entity": "glavobolom na levi strani"}, {"entity": "diplopijo"}, {"entity": "vrtoglavico"}, {"entity": "tinitusom"}, {"entity": "slušno okvaro"}] </s>'}

In [6]:
lengths = [len(prompt) for prompt in preprocessor.data['prompt']]
max_length = max(lengths)
max_length

1040